In [1]:
import numpy as np, pandas as pd, pickle

np.set_printoptions(linewidth=400)


def divide_datasets(num_of_constrains=6):
  data = pd.read_table('datasets/qws2.CSV', header=None, delimiter=',')
  pd_data = data.iloc[1:, :9].values.astype(float)
  item_size = pd_data.shape[0]
  data_dimensions = pd_data.shape[1]

  print(f'item_size: {item_size}\ndata_dimensions: {data_dimensions}')

  # 每一列的最大值和最小值
  column_max = np.max(pd_data, axis=0)
  column_min = np.min(pd_data, axis=0)
  # print(f'column_max: {column_max}\ncolumn_min: {column_min}')

  # 正负属性，如延迟越高越不好
  pos_or_neg = ['-', '+', '+', '+', '+', '+', '+', '-', '+']
  for (index, value) in enumerate(pos_or_neg):
    if value == '-':
      pd_data[:, index] = (pd_data[:, index] - column_min[index]) / (column_max[index] - column_min[index])
    else:
      pd_data[:, index] = (column_max[index] - pd_data[:, index]) / (column_max[index] - column_min[index])

  # 数据集的划分
  constrains_index = np.random.choice(item_size, num_of_constrains, replace=False)  # 随机选择6个索引
  constrains_service = pd_data[constrains_index, :]  # 选择6个索引的数据，作为约束集
  # print(f"constrains_service: \n{constrains_service}\nconstrains_service.shape: \n{constrains_service.shape}")

  candidates_service = np.delete(pd_data, constrains_index, axis=0)  # 删除6个索引的数据，作为候选集
  # print(f"candidates_service: \n{candidates_service}\ncandidates_service.shape: \n{candidates_service.shape}")

  return constrains_service, candidates_service


# 生成用户的服务调用历史
def generate_user_history(all_services):
  call_num = np.random.randint(10, 15 + 1)  # 随机生成10-15个服务调用
  call_history_index = np.random.choice(all_services.shape[0], call_num, replace=False)
  call_history_value = all_services[call_history_index - 1, :]
  return call_history_value


# 生成多个用户的服务调用历史记录
def generate_histories(num_of_users, all_services):
  histories = []
  for i in range(num_of_users):
    histories.append(generate_user_history(all_services))
  return histories


if __name__ == '__main__':
  constrains_service, candidates_service = divide_datasets()
  histories = generate_histories(10, candidates_service)
  print(f'constrains_service: \n{constrains_service}\nconstrains_service.shape: \n{constrains_service.shape}')
  print(f'candidates_service: \n{candidates_service}\ncandidates_service.shape: \n{candidates_service.shape}')
  [print(f'item_length:{len(item)}') for item in histories]

  with open('datasets/qws.pickle', 'wb') as f:
    pickle.dump(constrains_service, f)
    pickle.dump(candidates_service, f)
    pickle.dump(histories, f)
    f.close()


item_size: 2507
data_dimensions: 9
constrains_service: 
[[2.52893894e-02 2.36559140e-01 7.90697674e-01 2.39130435e-01 2.85714286e-01 1.64179104e-01 2.44444444e-01 1.69078042e-03 4.16666667e-02]
 [2.96809600e-02 0.00000000e+00 7.20930233e-01 0.00000000e+00 1.60714286e-01 3.28358209e-01 1.77777778e-01 1.81155045e-04 9.16666667e-01]
 [3.28634858e-01 6.23655914e-01 9.79069767e-01 6.19565217e-01 1.07142857e-01 1.64179104e-01 8.88888889e-02 3.22878674e-01 1.00000000e+00]
 [2.27554026e-01 1.29032258e-01 6.90697674e-01 4.34782609e-02 1.07142857e-01 3.28358209e-01 8.88888889e-02 2.59655564e-03 9.27083333e-01]
 [2.84694922e-02 2.15053763e-02 2.02325581e-01 1.08695652e-02 1.07142857e-01 0.00000000e+00 8.88888889e-02 1.87193546e-03 9.16666667e-01]
 [6.68326377e-01 1.82795699e-01 8.72093023e-01 1.73913043e-01 1.07142857e-01 1.64179104e-01 8.88888889e-02 4.22695104e-04 9.79166667e-01]]
constrains_service.shape: 
(6, 9)
candidates_service: 
[[5.36579259e-02 1.18279570e-01 8.37209302e-01 ... 3.3333333

In [2]:
import numpy as np, pickle, math, random
from itertools import combinations
from scipy import spatial
from scipy import stats
from sklearn.cluster import KMeans
from tqdm import tqdm

def get_thresholds(Candidates=None, alph=0.5):
  # 计算欧氏距离,余弦相似度,QoS相似度
  sim_dict = {}
  i = 0
  # 排列
  for Si, Sj in tqdm(list(combinations(range(len(Candidates)), 2))):
    temp_krcc, p_value = stats.kendalltau(Candidates[Si, :], Candidates[Sj, :])
    temp_dis = spatial.distance.euclidean(Candidates[Si, :], Candidates[Sj, :])
    temp_sim = (1 - alph) * (1.0 - temp_dis / math.sqrt(2.0)) + alph * temp_krcc
    sim_dict[(Si, Sj)] = temp_sim
    i = i + 1
    if i % 10000 == 0:
        pass
      # print(i)
  # thresh_sim=np.mean(list(sim_dict.values())) # average QoS similarity
  sim_arr = np.array(list(sim_dict.values()))
  thresh_sim = np.percentile(sim_arr, 80)
  return thresh_sim


if __name__ == '__main__':
  with open('datasets/qws.pickle', 'rb') as f:
    constrains_service = pickle.load(f)
    candidates_service = pickle.load(f)
    histories = pickle.load(f)
    f.close()
  print('tresholds:',get_thresholds(candidates_service))


  1%|          | 20485/3126250 [00:06<16:39, 3107.42it/s]


KeyboardInterrupt: 

In [4]:
!pwd

/home/lijiecn/Desktop/coding/vscode/fast-map-dpp/brandnew way
